# Customer Analytics: Preparing Data for Modeling

![Two data scientists working on a dashboard.](Students.png)

A common problem when creating models to generate business value from data is that the datasets can be so large that it can take days for the model to generate predictions. Ensuring that the dataset is stored as efficiently as possible is crucial for allowing these models to run on a more reasonable timescale without having to reduce the size of the dataset.

In this scenario, I have been hired by a major online data science training provider called *Training Data Ltd.* to clean up one of their largest customer datasets. This dataset will eventually be used to predict whether their students are looking for a new job or not, information that they will then use to direct them to prospective recruiters.

I have been given access to `customer_train.csv`, which is a subset of their entire customer dataset, so I can create a proof-of-concept of a much more efficient storage solution. They have also requested that the DataFrame should be filtered to only contain students with 10 or more years of experience at companies with at least 1000 employees, as their recruiter base is suited to more experienced professionals at enterprise companies.

The dataset contains anonymized student information, and whether they were looking for a new job or not during training:

| Column       | Description                                  |
|------------- |--------------------------------------------- |
| `student_id`   | A unique ID for each student.                 |
| `city`  | A code for the city the student lives in.  |
| `city_development_index` | A scaled development index for the city.       |
| `gender` | The student's gender.       |
| `relevant_experience` | An indicator of the student's work relevant experience.       |
| `enrolled_university` | The type of university course enrolled in (if any).       |
| `education_level` | The student's education level.       |
| `major_discipline` | The educational discipline of the student.       |
| `experience` | The student's total work experience (in years).       |
| `company_size` | The number of employees at the student's current employer.       |
| `last_new_job` | The number of years between the student's current and previous jobs.       |
| `training_hours` | The number of hours of training completed.       |
| `job_change` | An indicator of whether the student is looking for a new job (`1`) or not (`0`).       |

## Workflow/steps

- Columns containing integers will be stored as 32-bit integers (int32) and those containing floats stored as 16-bit floats (float16).


- Nominal categorical data columns are to be stored as the category data type while ordinal categorical data columns would be stored as ordered categories, with an order that reflects the natural order of the columns.


- The columns of ds_jobs_clean will be in the same order as the original dataset.


- Filter dataframe


In [17]:
# importing the pandas library
import pandas as pd

# loading the dataset
ds_jobs = pd.read_csv('customer_train.csv')

# making a copy of the dataframe for cleaning
ds_jobs_clean = ds_jobs.copy()

ds_jobs_clean

,student_id,city,city_development_index,gender,relevant_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,job_change
0,8949,city_103,0.920,Male,Has relevant experience,no_enrollment,Graduate,STEM,>20,NaN,NaN,1,36,1
1,29725,city_40,0.776,Male,No relevant experience,no_enrollment,Graduate,STEM,15,50-99,Pvt Ltd,>4,47,0
2,11561,city_21,0.624,NaN,No relevant experience,Full time course,Graduate,STEM,5,NaN,NaN,never,83,0
3,33241,city_115,0.789,NaN,No relevant experience,NaN,Graduate,Business Degree,<1,NaN,Pvt Ltd,never,52,1
4,666,city_162,0.767,Male,Has relevant experience,no_enrollment,Masters,STEM,>20,50-99,Funded Startup,4,8,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19153,7386,city_173,0.878,Male,No relevant experience,no_enrollment,Graduate,Humanities,14,NaN,NaN,1,42,1
19154,31398,city_103,0.920,Male,Has relevant experience,no_enrollment,Graduate,STEM,14,NaN,NaN,4,52,1
19155,24576,city_103,0.920,Male,Has relevant experience,no_enrollment,Graduate,STEM,>20,50-99,Pvt Ltd,4,44,0
19156,5756,city_65,0.802,Male,Has relevant experience,no_enrollment,High School,NaN,<1,500-999,Pvt Ltd,2,97,0


## Performing exploratory data anlysis

In [18]:
ds_jobs_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19158 entries, 0 to 19157
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   student_id              19158 non-null  int64  
 1   city                    19158 non-null  object 
 2   city_development_index  19158 non-null  float64
 3   gender                  14650 non-null  object 
 4   relevant_experience     19158 non-null  object 
 5   enrolled_university     18772 non-null  object 
 6   education_level         18698 non-null  object 
 7   major_discipline        16345 non-null  object 
 8   experience              19093 non-null  object 
 9   company_size            13220 non-null  object 
 10  company_type            13018 non-null  object 
 11  last_new_job            18735 non-null  object 
 12  training_hours          19158 non-null  int64  
 13  job_change              19158 non-null  int64  
dtypes: float64(1), int64(3), object(10)
me

In [19]:
# the number of null values per column
ds_jobs_clean.isna().sum().sort_values(ascending = False)

company_type              6140
company_size              5938
gender                    4508
major_discipline          2813
education_level            460
last_new_job               423
enrolled_university        386
experience                  65
student_id                   0
city                         0
city_development_index       0
relevant_experience          0
training_hours               0
job_change                   0
dtype: int64

### Nominal vs Ordinal data

Nominal data refers to categories or labels without any inherent order or ranking between them. For example, color = Red, Green, Blue. There is no inherent order or ranking between these categories, one isn't smaller or greater than the other.


On the flip side, Ordinal data represents categories or labels that have a meaningful order or ranking. E.g, Rating scales (1 star, 2 stars, 3 stars). Here we can say that one category is greater or lesser than another in terms of the variable being measured.


In [25]:
# Highlighting the nominal columns

nominal_columns = ['gender', 'major_discipline', 'city', 'company_type']

In [20]:
# Creating a dictionary of columns containing ordinal categorical data

ordered_cats = {
    'relevant_experience': ['No relevant experience', 'Has relevant experience'],
    'enrolled_university': ['no_enrollment', 'Part time course', 'Full time course'],
    'education_level': ['Primary School','High School', 'Graduate', 'Masters', 'Phd'],
    'experience': ['<1'] + list(map(str, range(1,21))) + ['>20'],
    'company_size': ['<10', '10-49', '100-499', '1000-4999','10000+', '50-99', '500-999', '5000-9999'],
    'last_new_job': ['never', '1', '2', '3', '4', '>4']
}

In [21]:
# next create a for loop to loop through the dataframe columns to efficiently change data types

for col in ds_jobs_clean:
    # converting from interger to int32
    if ds_jobs_clean[col].dtype == 'int':
        ds_jobs_clean[col] =  ds_jobs_clean[col].astype('int32')
    
    # converting from float to float16
    elif ds_jobs_clean[col].dtype == 'float':
        ds_jobs_clean[col] = ds_jobs_clean[col].astype('float16')
    
    # converting columns containing ordered categorical using the dictionary above
    elif col in ordered_cats.keys():
        category = pd.CategoricalDtype(ordered_cats[col], ordered = True)
        ds_jobs_clean[col] = ds_jobs_clean[col].astype(category)
      
    
    # converting the remaining nominal columns to a category data type
    else:
        ds_jobs_clean[col] = ds_jobs_clean[col].astype('category')

In [22]:
# To filter students with at least 10 years of experience at companies with at least a thousand employees

ds_jobs_clean = ds_jobs_clean[(ds_jobs_clean['experience'] >= '10') & (ds_jobs_clean['company_size'] >= '1000-4999')]

len(ds_jobs_clean)

4086

## To check and compare memory usage of the two dataframes

In [23]:
# Dataframe 1

ds_jobs_memory_usage = ds_jobs.memory_usage(deep = True).sum()

print(f'Total memory usage of the ds_jobs datframe is {ds_jobs_memory_usage} bytes')


Total memory usage of the ds_jobs datframe is 12391705 bytes


In [24]:
# Dataframe 2

ds_jobs_clean_memory_usage = ds_jobs_clean.memory_usage(deep = True).sum()

print(f'Total memory usage of the ds_jobs_clean datframe is {ds_jobs_clean_memory_usage} bytes')

Total memory usage of the ds_jobs_clean datframe is 811972 bytes


## Mission accomplished✅

Reducing the memory usage of this DataFrame involved optimizing the data types used to store its columns.


- By default, Pandas stores integer columns as 64-bit integers (int64) and float columns as 64-bit floats (float64), which consume more memory than necessary, especially for large datasets.By converting these columns to int32 and float16 respectively, I reduced the memory footprint of these columns while still preserving the data integrity within the precision limits of these data types.


- Nominal categories are stored as object data type by default. This again, consumes significant memory, especially since there were limited number of unique categories repeated across many rows. Converting these nominal categorical columns to the categorical data type helps Pandas to internally represent each unique category with an interger thereby reducing memory usage.


- When ordinal data is stored as regular categorical data - object - it doesn't preserve the order between categories. Changing the data type makes pandas maintain the order between categories to ensure that operations like sorting, comparison and even plotting of categorical plots are done correctly.